<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/OSI-SAF_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1b_OSI_SAF_SST_data_access_IFREMER_FTP.ipynb" target="_blank"><< Accessing Metop SST metagranule data through IFREMER opensearch queries</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_2_OSI_SAF_SST_file_structure.ipynb" target="_blank">Exploring OSI SAF SST product structure >></a>

<font color="#138D75">**EUMETSAT OSI SAF Training Service**</font> <br>
**Copyright:** 2024 EUMETSAT <br>
**License:** MIT

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fsensors%2Flearn-osi-saf-sst/HEAD?urlpath=%2Ftree%2F1_OSI_SAF_sst_introductory%2F1_1a_OSI_SAF_SST_data_access_IFREMER_Opensearch.ipynb"><img src="https://mybinder.org/badge_logo.svg" alt="Open in Binder"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>
</html>

<div class="alert alert-block alert-success">
<h3>Learn OSI SAF sea surface temperature: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **<a href="https://osi-saf.eumetsat.int/register" target="_blank">A EUMETSAT OSI SAF account</a>** if you are using or plan to use the EUMETSAT OSI SAF data.

There are no prerequisite notebooks for this module.
</div>
<hr>

# 1.1c Accessing Gobal Metop SST products throught the Eumetsat Data Store
### Data used 

| Dataset | EUMETSAT collection ID | OSI SAF website description | OSI SAF identifier |
|:-----------------:|:-----------------:|:-----------------:|:-----------------:|
| Global Metop Sea Surface Temperature | EO:EUM:DAT:METOP:GLB-SST-NC | <a href="https://osi-saf.eumetsat.int/products/osi-201-b" target="_blank">Description</a>| OSI-201-b |


### Learning outcomes

At the end of this notebook you will know;
* How to download different OSI SAF sea surface temperature (SST) products using the Eumetsat Data Store

<div class="alert alert-info" role="alert">

## <a id='TOC-TOP'></a>Contents

</div>
    
 1. [Creating the workspace](#section1)
 1. [Setting login information](#section2)
 1. [Defining satellite names and querry parameters](#section3)
 1. [Launch an OpenSearch request](#section4)

<hr>

<hr>

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Creating the workspace
[Back to top](#TOC-TOP)

</div>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [1]:
import os                       # a library that allows us access to basic operating system commands
import json                     # a library that helps us make JSON format files
import datetime                 # a libary that allows us to work with dates and times
import shutil                   # a library that allows us access to basic operating system commands like copy
import eumdac                   # a tool that helps us download via the eumetsat/data-store
import zipfile                  # a library that allow us to work with zip file
import requests                 # a library that allow us to work with HTTP
from pathlib import Path        # a library that helps construct system path objects

Next we will create a download directory to store the products we will download in this notebook.

In [2]:
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Setting login information
[Back to top](#TOC-TOP)

</div>

To access OSI SAF from the <a href="https://data.eumetsat.int">EUMETSAT Data Store</a>. we will use the EUMETSAT Data Access Client (`eumdac`). If you are working with the recommended Anaconda Python distribution and used the supplied environment file (environment.yml) to build this python environment (as detailed in the README), you will already have installed this. If not, you can install eumdac using;

`conda install -c eumetsat eumdac`

However, you can also find the source code on the <a href="https://gitlab.eumetsat.int/eumetlab/data-services/eumdac">EUMETSAT gitlab</a>. Please visit the EUMETSAT user support confluence spaces for the the <a href="https://eumetsatspace.atlassian.net/wiki/spaces/DSDS/overview">Data Store</a> and <a href="https://eumetsatspace.atlassian.net/wiki/spaces/EUMDAC/overview">eumdac</a> for more information.

In order to allow us to download data from the Data Store via API, we need to provide our credentials. We can do this in two ways; either by creating a file called `.eumdac/credentials` in our home directory (*option 1 - recommended*) or by supplying our credentials directly in this script (*option 2*). 

#### Option 1: creating  `.eumdac/credentials` file in our home directory

For most computer systems the home directory can be found at the path \user\username, /users/username, or /home/username depending on your operating system. In side this folder we need to create a folder called `.eumdac` (the dot is important) and within that a file called `credentials` (with no extension).

In this file we need to add the following information exactly as follows;

```
<your_consumer_key>,<your_consumer_secret>
```

You must replace `<your_consumer_key>` and `<your_consumer_secret>` with the information you extract from https://api.eumetsat.int/api-key/. You will need a <a href="https://eoportal.eumetsat.int/">EUMETSAT Earth Observation Portal account</a> to access this link, and in order to see the information you must click the "Show hidden fields" button at the bottom of the page. <b>To help with this step, we have made a supporting notebook that will create this file for you. You can find it <a href="../working-with-python/API_authentication.ipynb">here</a>, provided you cloned this repository with it's submodules, as detailed in the README.</b>

*Note: your key and secret are permanent, so you only need to do this once, but you should take care to never share them*

Once you have done this, you can read in your credentials using the commands in the following cell. These will be used to generate a time-limited token, which will refresh itself when it expires.

Before we use the Data Store to download data, we must first authenticate our access and retrieve an access token.

In [3]:
# load credentials
credentials_file = os.path.join(os.path.expanduser("~"),'.eumdac','credentials')
credentials = Path(credentials_file).read_text().split(',')
token = eumdac.AccessToken((credentials[0], credentials[1]))
print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)

This token '8708dd88-f01d-36de-b0e9-3c96044d591f' expires 2024-03-04 17:57:11.338754


#### Option 2: provide credentials directly

You can provide your credentials directly as follows;

`token = eumdac.AccessToken((consumer_key, consumer_secret))`

*Note: this method is convenient in the short term, but is not really recommended as you have to put your key and secret in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.*

Once we have a token (by either method described above) we can create a **datastore** 'object' that contains all the collections, and filter this list for those that are relevant to us. The code to the right of the colon is what will tell use the ID for the collection we are interested in.

In [4]:
# token = eumdac.AccessToken(("<your_consumer_key>", "<your_consumer_secret>"))
# print(f"This token '{token}' expires {token.expiration}")

# # create data store object
# datastore = eumdac.DataStore(token)

Note: this method is convenient in the short term, but is not really recommended as you have to put your provided username and password in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.

We will work with the Global Metop SST collection throughout this notebook. The code for this  is `EO:EUM:DAT:METOP:GLB-SST-NC`.

In [5]:
# set collection ID
collectionID = 'EO:EUM:DAT:METOP:GLB-SST-NC'

<div class="alert alert-info" role="alert">

## <a id='section3'></a>3. Filtering by time
[Back to top](#TOC-TOP)


## Filter by the lastest product

It is possible to search the most recent product of the collection by using the function `.first()`

In [6]:
# Time filter
selected_collection = datastore.get_collection(collectionID)
latest = selected_collection.search().first()
try:
    print(latest)
except eumdac.collection.CollectionError as error:
    print(f"Error related to the collection: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

S-OSI_-FRA_-MTOP-GLBSST_FIELD-202403040000Z


## Filter with start and end date
It is possible to select product from the collection filtering by time by adding a start and an end date to the function `.search(dtstart=date_start, dtend=end_date)`

In [7]:
# space/time filter the collection for products
selected_collection = datastore.get_collection(collectionID)
# Select start and end date for search
start = datetime.datetime(2023, 7, 15,0,0) # date format YYYY, MM, DD, HH, mm
end = datetime.datetime(2023, 7, 15, 4, 0)  # date format YYYY, MM, DD, HH, mm
products = selected_collection.search(geo=None, dtstart=start, dtend=end)
for product in products:
    try:
        print(product)
    except eumdac.collection.CollectionError as error:
        print(f"Error related to the collection: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")

S-OSI_-FRA_-MTOP-GLBSST_FIELD-202307150000Z


<div class="alert alert-info" role="alert">

## <a id='section4'></a>4. Download products
[Back to top](#TOC-TOP)

It is possible to download any component of a SAFE format file. This can be achieved by using iterating through the `entries` attribute of a product and string matching to your component of interest - in this case the manifest file. This method can be used to prune downloads for the enhanced, reduced or standard measurements only.

*Note: It is not currently possible to download a single variable from **inside** a netCDF file*.

In [8]:
download_dir = os.path.join(os.getcwd(), "products", "global")
os.makedirs(download_dir, exist_ok=True)

In [9]:
for product_id in products:
    print(f"Retrieving: {product_id}")    

    # selected_collection = datastore.get_collection(collection_id)
    selected_product = datastore.get_product(product_id=product_id, collection_id=collectionID)

    with selected_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
        print(f'Downloading {fsrc.name}.')
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of product {fsrc.name} finished.')

    with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.startswith(str(selected_product)):
                zip_ref.extract(file, download_dir)
        print(f'Unzipping of product {fdst.name} finished.')

    os.remove(fdst.name)

Retrieving: S-OSI_-FRA_-MTOP-GLBSST_FIELD-202307150000Z
Download of product S-OSI_-FRA_-MTOP-GLBSST_FIELD-202307150000Z.zip finished.
Unzipping of product /Users/benloveday/Desktop/CMTS_TEST/learn-osi-saf-sst/1_OSI_SAF_sst_introductory/products/global/S-OSI_-FRA_-MTOP-GLBSST_FIELD-202307150000Z.zip finished.


<hr>
<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1b_OSI_SAF_SST_data_access_IFREMER_FTP.ipynb" target="_blank"><< Accessing Metop SST metagranule data through IFREMER opensearch queries</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_2_OSI_SAF_SST_file_structure.ipynb" target="_blank">Exploring OSI SAF SST product structure >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>